In [ ]:
#import required libraries
from azure.ml import MLClient
from azure.ml.entities import CommandJob, Code, PipelineJob, Dataset, MPIConfiguration 

In [ ]:
#Enter details of your AML workspace
subscription_id = '15ae9cb6-95c1-483d-a0e3-b1a1a3b06324'
resource_group = 'balapv'
workspace = 'dpv2priprev'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(subscription_id, resource_group, workspace)

In [ ]:
#define the prep-job to run in the pipeline
prep_cmd = "python prep.py --raw_data ${{inputs.raw_data}} --prep_data ${{outputs.prep_data}}"
prep_data = Dataset(
    local_path="./data/"
)
prep_job_inputs = {'raw_data': prep_data}
prep_job_outputs = {'prep_data': None}

prep_job = CommandJob(
    code=Code(local_path="./src/prep"), #local path where the code is stored
    command = prep_cmd,
    inputs=prep_job_inputs,
    outputs=prep_job_outputs,
    environment = "AzureML-sklearn-0.24-ubuntu18.04-py37-cuda11-gpu:9",
    compute = "cpu-cluster"
)

In [ ]:
#lets create the pipeline
pipeline_job = PipelineJob(
    description = 'nyc-taxi-pipeline-example',
    jobs= {'prep-job':prep_job}
)

In [ ]:
#submit the pipeline job
returned_job = ml_client.jobs.create_or_update(pipeline_job)
#get a URL for the status of the job
returned_job.services["Studio"].endpoint